In [72]:
import numpy as np

In [ ]:
# load data

import cPickle
import gzip
 
with gzip.open('mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = cPickle.load(f)

In [381]:
class Layer:
    def get_parameters(self):
        return []
    
    def get_grads(self, input_,  next_grad):
        return []

class LinearLayer(Layer):
    
    def __init__(self, input_size, output_size):
        self.w = np.random.uniform(-1./output_size**0.5, 1./output_size**0.5, size=(output_size, input_size))
        self.b = np.zeros(output_size)

    def output(self, x):
        return np.dot(x, self.w.T) + self.b
    
    def grad(self, input_, grad_next_layer):
        return np.dot(grad_next_layer, self.w)
    
    def update_parameters(self, input_, learning_rate, grad_next_layer):
        self.w -= learning_rate * np.dot(input_.T, grad_next_layer)
        self.b -= learning_rate * grad_next_layer
        
    def get_grads(self, input_, grad_next_layer):
        return [np.dot(grad_next_layer.T, input_), grad_next_layer]
        
    def get_parameters(self):
        return [self.w, self.b]
    
class SigmoidLayer(Layer):
    def __init__(self):
        pass
    
    def sigmoid(self, x):
        return 1. / (1. + np.exp(x))
    
    def output(self, x):
        return self.sigmoid(x)
    
    def grad(self, input_, grad_next_layer):
        s = self.sigmoid(input_)
        # print 's', grad_next_layer.shape, s.shape
        return grad_next_layer * s * (1 - s)
        
class SoftmaxLayer(Layer):
    
    def __init__(self):
        pass
    
    def output(self, x):
        x_max = np.max(x, axis=1)
        probs = np.exp(x-x_max)
        return probs / np.sum(probs, axis=1)
    
    def grad(self, input_, grad_next_layer):
        sam = self.output(input_)
        return sam * (np.diagflat(np.ones(input_.shape[-1])) - np.array([np.roll(sam[0,:], i) for i in range(input_.shape[-1])])).T


In [383]:
epsilon = 10**-5

s = SoftmaxLayer()
x = np.random.uniform(0, 1, size=(1, 5))
grad = s.grad(x, 1)

for i in range(5):
    x[0, i] += epsilon
    plus = s.output(x)
    x[0, i] -= 2*epsilon
    moins = s.output(x)
    x[0, i] += epsilon
    
    print (plus - moins) / (2 * epsilon)
    print grad[:, i]

[[ 0.11620733 -0.01987407 -0.03567919 -0.03516075 -0.02549333]]
[ 0.11620733 -0.01987407 -0.03567919 -0.03516075 -0.02549333]
[[-0.01987407  0.12614338 -0.03935923 -0.03878731 -0.02812277]]
[-0.02812277  0.12819324 -0.02192393 -0.03935923 -0.03878731]
[[-0.03567919 -0.03935923  0.19515966 -0.06963345 -0.05048779]]
[-0.06963345 -0.05048779  0.23014067 -0.03935923 -0.0706602 ]
[[-0.03516075 -0.03878731 -0.06963345  0.19333567 -0.04975416]]
[-0.06963345 -0.06862162 -0.04975416  0.22679655 -0.03878731]
[[-0.02549333 -0.02812277 -0.05048779 -0.04975416  0.15385804]]
[-0.02812277 -0.05048779 -0.04975416 -0.03607429  0.16443901]


In [229]:
class MLP:
    
    def __init__(self, layers):
        self.layers = layers
        
    def train(self, n_epochs, batch_size, train, valid):
        train_x = train[0]
        train_y = train[1]
        n_batch = int(train_x.shape[0] / batch_size)
        
        for epoch in range(n_epochs):
            
            for current_batch in range(n_batch):
                fprop_results = self.fprop(train_x[current_batch*batch_size:(current_batch+1)*batch_size])
                grads = self.bprop(train_x[current_batch*batch_size:(current_batch+1)*batch_size],
                                   train_y[current_batch*batch_size:(current_batch+1)*batch_size],
                                  fprop_results)
                    
    def fprop(self, data_x, data_y):
        fprop_results = [data_x]
        # fprop
        for layer in self.layers:
            fprop_results.append(layer.output(fprop_results[-1]))
            
        cost = -np.log(fprop_results[-1][np.arange(fprop_results[-1].shape[0]), data_y]).mean()
        return cost, fprop_results
    
    def bprop(self, data_x, data_y, fprop_results):
        activations = fprop_results[-1]
        dc = -1 / activations * (np.arange(activations.shape[1])[:, None] == data_y).T

        grads = [dc]
        for index, layer in enumerate(self.layers[::-1]):
            grads.append(layer.grad(fprop_results[-index-1], grads[-1]))
            
        return grads

    def verify_gradients(self, input_size, n_classes):
    
        random_x = np.random.uniform(0, 1, size=(1, input_size))
        random_y = np.random.randint(10)
        epsilon = 10**-5
        
        cost, fprop_results = self.fprop(random_x, random_y)
        grads = self.bprop(random_x, random_y, fprop_results)
        
        for index in range(len(grads)):
            print index, fprop_results[index].shape, grads[-index-1].shape
        
        # compute gradients with finite difference
        for index, layer in enumerate(layers):
            for parameter, grad in zip(layer.get_parameters(), layer.get_grads(fprop_results[index], grads[-index-2])):
                
                numerical_estimate = np.zeros(parameter.shape)
                
                # in case the parameter is a matrix
                if len(parameter.shape) == 2:
                    for i in range(parameter.shape[0]):
                        for j in range(parameter.shape[1]):
                            parameter[i, j] += epsilon
                            cost_right, discard = self.fprop(random_x, random_y)
                            parameter[i, j] -= 2 * epsilon
                            cost_left, discard = self.fprop(random_x, random_y)
                            parameter[i, j] += epsilon
                            numerical_estimate[i, j] = (cost_right - cost_left) / (2 * epsilon)
                            
                # in case the parameter is a vector
                if len(parameter.shape) == 1:
                    numerical_estimate = np.zeros(parameter.shape)
                    for i in range(parameter.shape[0]):
                        parameter[i] += epsilon
                        cost_right, discard = self.fprop(random_x, random_y)
                        parameter[i] -= 2 * epsilon
                        cost_left, discard = self.fprop(random_x, random_y)
                        parameter[i] += epsilon
                        numerical_estimate[i] = (cost_right - cost_left) / (2 * epsilon)
                            
                print np.allclose(numerical_estimate, grad)
                print np.abs(numerical_estimate- grad).mean()
    
    def predict(self):
        pass

In [384]:
# build actual mlp

layers = [LinearLayer(100, 100), SigmoidLayer(), LinearLayer(100, 10), SoftmaxLayer()]
mlp = MLP(layers)

mlp.verify_gradients(100, 10)
# mlp.train(5, 100, train_set, valid_set)

0 (1, 100) (10, 100)
1 (1, 100) (10, 100)
2 (1, 100) (10, 100)
3 (1, 10) (10, 10)
4 (1, 10) (1, 10)


ValueError: shapes (100,10) and (1,100) not aligned: 10 (dim 1) != 1 (dim 0)

In [264]:
a = np.array([[1, 2, 3, 4]])
b = a.repeat(4, axis=0)

b[range(b.shape[0]), range(b.shape[0])] -= np.array(range(4))[None,:]
b

array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4]])

In [349]:
a = np.array([1, 2, 3])
print np.roll(a, 2)
print a

[2 3 1]
[1 2 3]
